In [14]:
!pip install langchain


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ssetr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
     ---------------------------------------- 83.5/83.5 kB 2.4 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
   ---------------------------------------- 811.8/811.8 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 365.2/365.2 kB 1.6 MB/s eta 0:00:00
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
   ---------------------------------------- 1.6/1.6 MB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 239.4/239.4 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 55.4/55.4 kB 2.8 MB/s eta 0:00:00
   

In [58]:
!pip install pypdf


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ssetr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 284.0/284.0 kB 1.0 MB/s eta 0:00:00


In [56]:
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os 
import requests
from langchain.document_loaders.sitemap import SitemapLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
with open('url_mapping.json', 'r', encoding='utf-8') as file:
    url_mapping = json.load(file)

In [28]:
def webscrap(sitemap_url = "https://www.newhaven.edu/sitemap.xml"):
    response = requests.get(sitemap_url)
    keywords = [
        "data-science",
        "Artificial",
        "intelligence",
        "faq",
        "tagliatela",
        "international",
        "immigration",
        "visa",
        "international-services",
        "course",
        "bursars",
        "one-stop",
        "registrar",
        "career",
        "technical-support",
        "library",
        "counseling",
        "career"

    ]
    soup = BeautifulSoup(response.content, 'xml')
    urls = [element.text for element in soup.find_all('loc')]
    urls = [url for url in urls if any(keyword.lower() in url.lower() for keyword in keywords)]
    pdf_urls = [url for url in urls if url.lower().endswith('.pdf')]
    php_urls = [url for url in urls if url.lower().endswith('.php')]
    return pdf_urls, php_urls

In [29]:
def is_valid_url(url):
    """
    Checks if the URL is a valid HTTP or HTTPS URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [30]:
def download_pdf(url, folder="PDFs"):
    """
    Downloads a single PDF file from a URL.
    """
    if not os.path.isdir(folder):
        os.makedirs(folder)

    try:
        response = requests.get(url, stream=True)
        
        if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
            filename = os.path.join(folder, url.split('/')[-1] or "downloaded_pdf.pdf")

            with open(filename, 'wb') as file:
                file.write(response.content)
            print(f"Successfully downloaded {url}")
        else:
            print(f"Failed to download {url} - Response code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

In [31]:
def loadDataFrmPhpUrls(php_urls):
    try:
        sitemap_loader = SitemapLoader(web_path="https://www.newhaven.edu/sitemap.xml",
                                    filter_urls=php_urls)
        docs = sitemap_loader.load()
        print("Php Data Loaded successfully")
    except Exception as e:
        print(f"An error occurred: {e}")
    return docs

In [65]:
def loadPDFData():
    pdf_data = []
    cwd = os.getcwd()
    for pdf in os.listdir('PDFs'):
        pdf_path = os.path.join(cwd, 'PDFs')
        pdf = os.path.join(pdf_path,pdf)
        loader = PyPDFLoader(pdf)
        pages = loader.load_and_split()
        for page in pages:
            print (page.metadata['source'])
            source = page.metadata['source'].split('\\')[-1]
            print(source)
            page.metadata['source'] = url_mapping[source]
        pdf_data.extend(pages)
    print("PDF Data Loaded successfully")
    return pdf_data

In [33]:
def split_data_chunks(final_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1200,
        chunk_overlap  = 200,
        length_function = len,
    )
    docs_chunks = text_splitter.split_documents(final_data)
    print("Splitting Data Done Successfully")
    return docs_chunks

In [48]:

pdf_urls,php_urls = webscrap()

In [35]:
print("PHP URLs:",len(php_urls))

PHP URLs: 216


In [49]:
print("PDF URLs:",len(pdf_urls))

PDF URLs: 158


In [53]:
for url in pdf_urls:
    download_pdf(url, folder="PDFs")

Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/2019-soar-families.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/enrolling-in-direct-deposit.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/enroll-in-monthly-payment-plan.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/epay-quick-guide.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/how-to-create-authorized-payer.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/how-to-make-payment-verto-experience.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/international-booklet-faq.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/about/departments/one-stop/soar-pr

Successfully downloaded https://www.newhaven.edu/_resources/documents/academics/study-abroad/program-worksheets/international-dev-and-poli-sci.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/admissions/graduate/international/faq-international-students-fall-2020.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/admissions/graduate/international/financial-statement-of-international-students-graduate.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/admissions/graduate/international/graduate-international-viewbook.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/admissions/graduate/international/sevis-transfer.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/admissions/graduate/salary-data/clinical-mental-health-counseling-ma.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/admissions/graduate/salary-data/data-science-ms.pdf
Failed to downloa

Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/fall-newsletter-2018.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2006newsletter.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2008newsletter.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2009newsletter.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2010newsletter.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2011newsletter.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2012newsletter.pdf
Successfully downloaded https://www.newhaven.edu/_resources/documents/library/general/newsletters/spring2013newsletter.pdf
Successfully dow

In [66]:
pdf_data = loadPDFData()

C:\Users\ssetr\Desktop\Capstone\PDFs\00139700_HEERF_Q3202_070122.pdf
00139700_HEERF_Q3202_070122.pdf


KeyError: '00139700_HEERF_Q3202_070122.pdf'

In [39]:
php_data = loadDataFrmPhpUrls(php_urls)

An error occurred: asyncio.run() cannot be called from a running event loop


C:\Users\ssetr\AppData\Local\Temp\ipykernel_23704\3262413682.py:8: RuntimeWarning: coroutine 'WebBaseLoader.fetch_all' was never awaited
  print(f"An error occurred: {e}")


UnboundLocalError: local variable 'docs' referenced before assignment

In [67]:
# final_data = []
# final_data.extend(pdf_data)

In [68]:
# final_data

[]

In [69]:
# data = split_data_chunks(final_data)

Splitting Data Done Successfully


In [70]:
# pdf_data

[]

In [23]:
# async def main():
#     print(1)
    
# await main()

1
